In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw

Using TensorFlow backend.


In [2]:
#usefull functions

def paring(mom,dad):
    return(np.concatenate((mom[:3], dad[3:]), axis=0))

def selected(maxis):
    childs= []
   
    indices = maxis.argsort()[-2:][::-1]
    parents= []
    for ind in indices:
        r = 0
        for k in genomes[ind]:
         
            k=list(k)
            parents.append(k)
            r = r+1
   
    
    random.shuffle(parents)
    print(parents)
    f= 0
    while f < len(parents)-1: 
        mom = parents[f]
        f= f +1
        dad = parents[f]     
        child = paring(mom,dad)
        childs.append(child)
        child = paring(dad,mom)
        childs.append(child)
        
        
    return childs
        

In [4]:
#genomes as a list of gene sequences
genomes =[]
for i in range(4):
    genomes.append([np.random.randint(2, size=6)]) # add 5 randomly initialised indiviudals

    
print (genomes)

accuracy_of_population= []

[[array([0, 0, 1, 1, 0, 0])], [array([1, 1, 0, 1, 1, 0])], [array([1, 0, 1, 0, 1, 0])], [array([1, 0, 1, 1, 1, 0])]]


In [ ]:



for element in genomes:
    for indiv in element:
        print(indiv)
        




    batch_size = 128
    num_classes = 10
    epochs = 12

    # input image dimensions
    img_rows, img_cols = 28, 28

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()




    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    ########

    for gen in indiv:
        if (gen== 0):
            model.add(Conv2D(64, (3, 3), activation='relu'))
        elif (gen == 1):
            model.add(Dense(128, activation='relu'))




    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    accuracy_of_population.append(score[1])

print(accuracy_of_population)

[0 0 1 1 0 0]


W0911 21:19:10.821372 17252 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0911 21:19:10.860167 17252 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0911 21:19:10.874555 17252 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0911 21:19:10.937589 17252 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0911 21:19:10.941583 17252 deprecation_wrapper.py:119] From C:\Users\vivia\Ana

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


W0911 21:19:11.265273 17252 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0911 21:19:11.279143 17252 deprecation_wrapper.py:119] From C:\Users\vivia\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0911 21:19:11.479817 17252 deprecation.py:323] From C:\Users\vivia\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 39s 657us/step - loss: 0.3709 - acc: 0.8799 - val_loss: 0.0800 - val_acc: 0.9760
Epoch 2/12
60000/60000 [==============================] - 32s 526us/step - loss: 0.0798 - acc: 0.9757 - val_loss: 0.0380 - val_acc: 0.9879
Epoch 3/12
60000/60000 [==============================] - 31s 520us/step - loss: 0.0502 - acc: 0.9848 - val_loss: 0.0321 - val_acc: 0.9898
Epoch 4/12
60000/60000 [==============================] - 32s 540us/step - loss: 0.0415 - acc: 0.9870 - val_loss: 0.0229 - val_acc: 0.9921
Epoch 5/12
60000/60000 [==============================] - 30s 508us/step - loss: 0.0347 - acc: 0.9895 - val_loss: 0.0296 - val_acc: 0.9909
Epoch 6/12
60000/60000 [==============================] - 32s 533us/step - loss: 0.0296 - acc: 0.9910 - val_loss: 0.0222 - val_acc: 0.9934
Epoch 7/12
60000/60000 [==============================] - 32s 542us/step - loss: 0.0260 - acc: 0.9916 - val_loss: 0.

In [ ]:
print("NEW Population :")
print(selected(accuracy_of_population))